In [ ]:
import pandas as pd
from sqlalchemy import create_engine
# pip install python-binance
from binance.client import Client
from binance import BinanceSocketManager

In [ ]:
client = Client()
bsm = BinanceSocketManager(client)

socket = bsm.kline_socket('BTCUSDT')
await socket.__aenter__()
msg = await socket.recv()

print(msg)
await socket.__aexit__(None, None, None)

In [ ]:
def datatransfo(msg):
    df = pd.DataFrame({'Time':msg['E'], 'Price':msg['k']['c']}, index=[0])
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [ ]:
datatransfo(msg)

In [ ]:
engine = create_engine('sqlite:///BTCUSDTstream.db')

In [ ]:
current_event = pd.Series(pd.to_datetime(0))
while True:
    await socket.__aenter__()
    msg = await socket.recv()
    await socket.__aexit__(None, None, None)
    df = datatransfo(msg)
    if df.Time.values > current_event.values:
        current_event = df.Time
        df.to_sql('BTCUSDT', engine, if_exists='append', index=False)
